## Import necessary libraries

In [ ]:
import ...

## Read data

In [ ]:
data = ...

In [ ]:
data

## stop words

In [ ]:
...

## split data

In [ ]:
X_train, X_test, y_train, y_test = ...

## Define ML pipeline

You can use sklearn's TfidfVectorizer that combines CountVectorizer and TfidfTransformer in a single step.

Set ngram_range to (1,1) for outputting only one-word tokens, (1,2) for one-word and two-word tokens, (2, 3) for two-word and three-word tokens, etc.

ngram_range works hand-in-hand with analyzer. Set analyzer to "word" for outputting words and phrases, or set it to "char" to output character ngrams.

If you want your output to have both "word" and "char" features, use sklearn's FeatureUnion.

Since stop words generally have a high frequency, it might make sense to use max_df as a float of say 0.95 to remove the top 5% but then you're assuming that the top 5% is all stop words which might not be the case. It really depends on your text data. In some lines of work, it's very common that the top words or phrases are NOT stop words because you work with dense text (search query data) in very specific topics.

Use min_df as an integer to remove rare-occurring words. If they only occur once or twice, they won't add much value and are usually really obscure. Furthermore, there's generally a lot of them so ignoring them with say min_df=5 can greatly reduce your memory consumption and data size.

token_pattern allows you to use a regex pattern e.g. \b\w\w+\b which means that tokens have to be at least 2 characters long so words like "I", "a" are removed and also numbers like 0 - 9 are removed. You'll also notice that this example removes apostrophes.

In [ ]:
pipeline = ...

## Train model

In [ ]:
...

## Score model

In [ ]:
...

## Save model

In [ ]:
# save the model as 'model.pkl'